# Hidden Markov Model (HMM)



A Hidden Markov Model (HMM) is a statistical model for sequential data where unobservable states (e.g., underlying ancestry) evolve over time according to a Markov chain, and at each time point we observe noisy or indirect measurements that depend on the current hidden state.


# Graphical Summary

![fig](./cartoons/fig.svg)

# Key Formula

The complete probability of any sequence of hidden states and observations in an HMM is given by:

$$
\mathbb{P}(z_{1:T}, x_{1:T}) = \mathbb{P}(z_1) \prod_{t=2}^{T} \mathbb{P}(z_t | z_{t-1}) \prod_{t=1}^{T} \mathbb{P}(x_t | z_t)
$$

- $z_t$ = hidden state at time $t$ (what's happening behind the scenes)
- $x_t$ = observation at time $t$ (what you can actually see)
- $z_{1:T}$ = sequence of hidden states from time 1 to $T$
- $x_{1:T}$ = sequence of observations from time 1 to $T$
- $T$ = total number of time steps


# Technical Details

## Markov Chain

A Markov chain is a sequence of random variables $z_1, z_2, \ldots, z_T$ where the future state depends only on the current state, not on the history:

$$
\mathbb{P}(z_{t+1} | z_1, z_2, \ldots, z_t) = \mathbb{P}(z_{t+1} | z_t)
$$

This is the **Markov property**. The chain is fully specified by initial distribution $\mathbb{P}(z_1)$ and transition probabilities $\mathbb{P}(z_{t+1} | z_t)$.

An HMM extends this by adding observations: the states $z_t$ are hidden (unobservable), but at each time $t$ we observe $x_t$ that depends only on the current hidden state $z_t$. This is called **output independence**.

## Inference Problem

Given a sequence of observations $x_{1:T}$, we want to infer the hidden states. Specifically, we want to compute the posterior distribution $\mathbb{P}(z_t | x_{1:T})$ for each time $t$. The forward-backward algorithm solves this efficiently using dynamic programming.

## Forward Probability

The forward probability accumulates information from past observations.

**Definition:**

$$
\alpha_t(k) := \mathbb{P}(z_t = k, x_{1:t})
$$

Joint probability of state $k$ at time $t$ and all past observations.

**Forward Recursion:**

$$
\alpha_{t+1}(k) = \mathbb{P}(x_{t+1} | z_{t+1} = k) \sum_{j} \alpha_t(j) \cdot \mathbb{P}(z_{t+1} = k | z_t = j)
$$

## Backward Probability

The backward probability accumulates information from future observations.

**Definition:**

$$
\beta_t(k) := \mathbb{P}(x_{t+1:T} | z_t = k)
$$

Probability of all future observations given state $k$ at time $t$.

**Backward Recursion:**

$$
\beta_t(k) = \sum_{j} \mathbb{P}(z_{t+1} = j | z_t = k) \cdot \mathbb{P}(x_{t+1} | z_{t+1} = j) \cdot \beta_{t+1}(j)
$$

## Posterior Distribution

By the Markov property, conditioning on $z_t$ makes past and future observations independent:

$$
\mathbb{P}(z_t = k, x_{1:T}) = \mathbb{P}(z_t = k, x_{1:t}) \cdot \mathbb{P}(x_{t+1:T} | z_t = k) = \alpha_t(k) \cdot \beta_t(k)
$$

Thus, the posterior distribution is:

$$
\mathbb{P}(z_t = k | x_{1:T}) = \frac{\alpha_t(k) \cdot \beta_t(k)}{\sum_{k'} \alpha_t(k') \cdot \beta_t(k')}
$$

This answers: "Given all observations, what is the probability we were in state $k$ at time $t$?"

# Example 1: ELAI (Efficient Local Ancestry Inference)

## Background

When two populations mix (e.g., through migration and intermarriage), their descendants carry **chromosomal segments** from both ancestral populations. Local ancestry inference aims to determine, for each segment of an admixed individual's genome, which ancestral population it originated from.


ELAI uses a **two-layer Hidden Markov Model** to capture two scales of genetic variation:

1. **Upper Layer (Coarse scale)**: Which **ancestral POPULATION** does a genomic segment come from?
   - Example: European vs. African vs. Native American

2. **Lower Layer (Fine scale)**: Which specific **ancestral HAPLOTYPE** within that population?
   - Example: Which of the many European haplotypes?

This hierarchical structure captures:
- **Admixture LD**: Linkage disequilibrium between populations (spans 1-10 cM)
- **Within-population LD**: Linkage disequilibrium within populations (spans 0.1-1 cM)


## Data Generation

First, we create reference populations with different allele frequency patterns.


In [ ]:
rm(list = ls())
set.seed(720)
# Parameters
n_markers <- 50          # Number of SNP markers
n_ref_haps_per_pop <- 10 # Reference haplotypes per population

# Population 1 (e.g., European): lower allele frequencies
# Make populations VERY distinct for clear inference
pop1_freq <- runif(n_markers, 0.05, 0.25)

# Population 2 (e.g., African): higher allele frequencies
pop2_freq <- runif(n_markers, 0.75, 0.95)

cat("Average allele frequency difference between populations:", 
    round(mean(pop2_freq - pop1_freq), 2), "\n")
cat("This large difference makes ancestry inference easier.\n\n")

# Generate reference haplotypes
ref_pop1 <- matrix(
  rbinom(n_markers * n_ref_haps_per_pop, 1, prob = rep(pop1_freq, n_ref_haps_per_pop)),
  nrow = n_markers,
  ncol = n_ref_haps_per_pop
)

ref_pop2 <- matrix(
  rbinom(n_markers * n_ref_haps_per_pop, 1, prob = rep(pop2_freq, n_ref_haps_per_pop)),
  nrow = n_markers,
  ncol = n_ref_haps_per_pop
)

In [ ]:
# True ancestry (upper layer): which population at each marker
true_ancestry <- rep(1, n_markers)
true_ancestry[11:30] <- 2  # Larger middle segment from population 2
true_ancestry[41:48] <- 2  # Larger second segment from population 2

# Generate observed haplotype by copying from appropriate population
observed_haplotype <- numeric(n_markers)

for (m in 1:n_markers) {
  if (true_ancestry[m] == 1) {
    # Copy from a random haplotype in population 1
    donor_hap <- sample(1:n_ref_haps_per_pop, 1)
    observed_haplotype[m] <- ref_pop1[m, donor_hap]
  } else {
    # Copy from a random haplotype in population 2
    donor_hap <- sample(1:n_ref_haps_per_pop, 1)
    observed_haplotype[m] <- ref_pop2[m, donor_hap]
  }
}

cat("\nAdmixed individual created with ancestry blocks:\n")
cat(sprintf("  Markers 1-10:   Population %d\n", true_ancestry[1]))
cat(sprintf("  Markers 11-30:  Population %d\n", true_ancestry[11]))
cat(sprintf("  Markers 31-40:  Population %d\n", true_ancestry[31]))
cat(sprintf("  Markers 41-48:  Population %d\n", true_ancestry[41]))
cat(sprintf("  Markers 49-50:  Population %d\n", true_ancestry[49]))

**Key Point**: This simulates **recombination history** - the admixed individual's chromosome is a mosaic created by recombination events during generations of admixture.

## Model Structure and Parameters

In [ ]:
# UPPER LAYER: Ancestral populations
S <- 2  # Number of populations
upper_clusters <- 1:S

# LOWER LAYER: Ancestral haplotypes within each population
K <- 4  # Total number of lower-layer clusters (2 per population for simplicity)

# Admixture proportion - based on true ancestry created earlier
true_admix <- c(mean(true_ancestry == 1), mean(true_ancestry == 2))
admix_proportion <- true_admix  # Use true proportions for better inference

### 1. Allele Frequencies (θ_mk)

Each lower-layer cluster has its own allele frequency at each marker.

In [ ]:
# Cluster allele frequencies (theta in ELAI notation)
# Clusters 1-2: similar to population 1
# Clusters 3-4: similar to population 2
# Use very small noise to keep clusters very close to population frequencies
theta <- matrix(0, nrow = n_markers, ncol = K)
theta[, 1] <- pop1_freq + rnorm(n_markers, 0, 0.01)
theta[, 2] <- pop1_freq + rnorm(n_markers, 0, 0.01)
theta[, 3] <- pop2_freq + rnorm(n_markers, 0, 0.01)
theta[, 4] <- pop2_freq + rnorm(n_markers, 0, 0.01)
theta <- pmax(pmin(theta, 0.95), 0.05)  # Keep in valid range [0.05, 0.95]

# Show first 5 markers
cat("Allele frequencies (first 5 markers):\n")
dim(theta)
print(round(theta[1:5, ], 3))


**Key Point**: θ_mk represents the probability that ancestral haplotype k carries allele "1" at marker m.

### 2. Beta (β_msk): Connection Between Layers

β_msk is the probability that lower-layer cluster k belongs to upper-layer cluster s.


In [19]:
# Beta: probability that lower cluster k belongs to upper cluster s
beta <- matrix(0, nrow = S, ncol = K)
beta[1, 1:2] <- c(0.6, 0.4)  # Pop 1 uses clusters 1 and 2
beta[2, 3:4] <- c(0.5, 0.5)  # Pop 2 uses clusters 3 and 4

cat("Beta matrix (connection between layers):\n")
cat("Rows = populations, Columns = lower clusters\n")
print(beta)

Beta matrix (connection between layers):
Rows = populations, Columns = lower clusters
     [,1] [,2] [,3] [,4]
[1,]  0.6  0.4  0.0  0.0
[2,]  0.0  0.0  0.5  0.5


**Key Point**: Beta connects the two layers. It says "if you're in population 1, you'll likely use lower clusters 1 or 2."

### 3. Transition Probabilities

In [ ]:
rho <- 0.03     # Lower-layer switch probability (within-population recombination)
                # Small value allows stable ancestry blocks
j_prob <- 0.01  # Upper-layer switch probability (ancestry switch)
                # Very small to match the rare ancestry switches in the data

cat(sprintf("Transition probabilities:\n"))
cat(sprintf("  ρ (rho) = %.2f  [Lower layer switch - recombination within population]\n", rho))
cat(sprintf("  j       = %.2f  [Upper layer switch - ancestry change]\n", j_prob))

**Key Point**: 
- **j** controls how often ancestry switches between populations
- **ρ** controls recombination within a population (switching between ancestral haplotypes)

## HMM Inference

### Emission Probability: Equation (1)

The emission probability models how observed alleles are generated from hidden states.

$$
p(h_m | Y_m = k, \theta) = \begin{cases} 
\theta_{mk} & \text{if } h_m = 1 \\
1 - \theta_{mk} & \text{if } h_m = 0
\end{cases}
$$

In [21]:
# Emission probability function
emission_prob <- function(h_m, k, m) {
  # h_m: observed allele (0 or 1)
  # k: lower-layer cluster
  # m: marker position
  if (h_m == 1) {
    return(theta[m, k])
  } else {
    return(1 - theta[m, k])
  }
}

# Example: probability of observing allele 1 at marker 1 from cluster 1
cat(sprintf("Example: P(observe '1' at marker 1 | cluster 1) = %.3f\n", 
    emission_prob(1, 1, 1)))
cat(sprintf("Example: P(observe '0' at marker 1 | cluster 1) = %.3f\n", 
    emission_prob(0, 1, 1)))

Example: P(observe '1' at marker 1 | cluster 1) = 0.279
Example: P(observe '0' at marker 1 | cluster 1) = 0.721


**Key Point**: Once you know which ancestral haplotype (lower cluster) you're copying from, the allele follows a simple Bernoulli distribution.

### Transition Probability: Equation (4)

This is the heart of ELAI. The transition has **three terms**:

$$P(X_m=s, Y_m=k | X_{m-1}=s', Y_{m-1}=k') = $$
$$\underbrace{j \cdot a_s \cdot \beta_{sk}}_{\text{Term 1: Pop switch}} + 
\underbrace{(1-j) \cdot \rho \cdot \beta_{sk} \cdot I(s=s')}_{\text{Term 2: Hap switch}} + 
\underbrace{(1-j) \cdot (1-\rho) \cdot I(s=s') \cdot I(k=k')}_{\text{Term 3: No switch}}$$

In [22]:
# Transition probability function
transition_prob <- function(s, k, s_prev, k_prev, m) {
  # s, k: current state (upper cluster, lower cluster)
  # s_prev, k_prev: previous state
  # m: marker position
  
  # Term 1: Upper layer switches (both layers must change)
  term1 <- j_prob * admix_proportion[s] * beta[s, k]
  
  # Term 2: Only lower layer switches (upper stays same)
  term2 <- (1 - j_prob) * rho * beta[s, k] * (s == s_prev)
  
  # Term 3: No switches (stay in same state)
  term3 <- (1 - j_prob) * (1 - rho) * (s == s_prev) * (k == k_prev)
  
  return(term1 + term2 + term3)
}

# Example: staying in state (1,1)
cat(sprintf("Example: P(stay in state (s=1, k=1)) = %.4f\n", 
    transition_prob(1, 1, 1, 1, 2)))

# Example: switching populations
cat(sprintf("Example: P(switch from (s=1, k=1) to (s=2, k=3)) = %.4f\n", 
    transition_prob(2, 3, 1, 1, 2)))

Example: P(stay in state (s=1, k=1)) = 0.9330
Example: P(switch from (s=1, k=1) to (s=2, k=3)) = 0.0075


**Key Point**: The three terms represent:
1. **Ancestry switch**: Change populations (recombination between ancestral populations)
2. **Haplotype switch**: Stay in same population but switch haplotypes (recombination within population)
3. **No switch**: Continue copying from same haplotype

### Forward Algorithm

The forward probability $\alpha_m(s,k) = P(h_{1:m}, X_m=s, Y_m=k)$ represents the probability of observing markers 1 to m AND being in state (s,k) at marker m.

In [23]:
# Initialize forward probabilities
forward <- array(0, dim = c(n_markers, S, K))

# Base case: marker 1
for (s in 1:S) {
  for (k in 1:K) {
    forward[1, s, k] <- admix_proportion[s] * beta[s, k] * 
                        emission_prob(observed_haplotype[1], k, 1)
  }
}

cat("Forward algorithm - initialization at marker 1:\n")
cat("State (s=1, k=1):", forward[1, 1, 1], "\n")
cat("State (s=2, k=3):", forward[1, 2, 3], "\n")

# Forward recursion
for (m in 2:n_markers) {
  for (s in 1:S) {
    for (k in 1:K) {
      # Sum over all previous states
      sum_val <- 0
      for (s_prev in 1:S) {
        for (k_prev in 1:K) {
          sum_val <- sum_val + 
            forward[m-1, s_prev, k_prev] * 
            transition_prob(s, k, s_prev, k_prev, m)
        }
      }
      forward[m, s, k] <- emission_prob(observed_haplotype[m], k, m) * sum_val
    }
  }
  # Normalize to prevent underflow
  forward[m, , ] <- forward[m, , ] / sum(forward[m, , ])
}

cat("\nForward algorithm completed for all", n_markers, "markers\n")

Forward algorithm - initialization at marker 1:
State (s=1, k=1): 0.3028165 
State (s=2, k=3): 0.04059814 

Forward algorithm completed for all 50 markers


**Key Point**: Forward probabilities are computed recursively - each state's probability depends on all previous states, weighted by transition probabilities.

### Backward Algorithm

The backward probability $\beta_m(s,k) = P(h_{m+1:M} | X_m=s, Y_m=k)$ represents the probability of observing markers (m+1) to M GIVEN we're in state (s,k) at marker m.

In [24]:
# Initialize backward probabilities
backward <- array(1, dim = c(n_markers, S, K))

# Backward recursion
for (m in (n_markers-1):1) {
  for (s in 1:S) {
    for (k in 1:K) {
      sum_val <- 0
      for (s_next in 1:S) {
        for (k_next in 1:K) {
          sum_val <- sum_val + 
            transition_prob(s_next, k_next, s, k, m+1) *
            emission_prob(observed_haplotype[m+1], k_next, m+1) *
            backward[m+1, s_next, k_next]
        }
      }
      backward[m, s, k] <- sum_val
    }
  }
  # Normalize to prevent underflow
  backward[m, , ] <- backward[m, , ] / sum(backward[m, , ])
}

cat("Backward algorithm completed for all", n_markers, "markers\n")

Backward algorithm completed for all 50 markers


**Key Point**: The backward algorithm processes the data in reverse, allowing us to combine information from both directions when computing posterior probabilities.

## Results

### Posterior Probabilities

Combining forward and backward probabilities gives the posterior: $P(X_m=s | \text{all data})$

In [25]:
# Compute posterior probability: P(X_m = s | observed data)
posterior_ancestry <- matrix(0, nrow = n_markers, ncol = S)

for (m in 1:n_markers) {
  for (s in 1:S) {
    # Sum over all lower clusters k
    for (k in 1:K) {
      posterior_ancestry[m, s] <- posterior_ancestry[m, s] + 
                                  forward[m, s, k] * backward[m, s, k]
    }
  }
  # Normalize
  posterior_ancestry[m, ] <- posterior_ancestry[m, ] / sum(posterior_ancestry[m, ])
}

# Most likely ancestry at each marker
inferred_ancestry <- apply(posterior_ancestry, 1, which.max)

# Calculate accuracy
accuracy <- sum(inferred_ancestry == true_ancestry) / n_markers

cat("\n=================================================================\n")
cat("LOCAL ANCESTRY INFERENCE RESULTS\n")
cat("=================================================================\n")
cat(sprintf("\nAccuracy: %.1f%% of markers correctly assigned\n", accuracy * 100))

# Estimated vs true admixture proportions
estimated_admix <- colMeans(posterior_ancestry)
cat("\nGlobal Admixture Proportions:\n")
cat(sprintf("  Population 1: True = %.2f, Estimated = %.2f\n", 
            mean(true_ancestry == 1), estimated_admix[1]))
cat(sprintf("  Population 2: True = %.2f, Estimated = %.2f\n", 
            mean(true_ancestry == 2), estimated_admix[2]))


LOCAL ANCESTRY INFERENCE RESULTS

Accuracy: 56.0% of markers correctly assigned

Global Admixture Proportions:
  Population 1: True = 0.56, Estimated = 0.87
  Population 2: True = 0.44, Estimated = 0.13


**Key Point**: The posterior probability represents our **confidence** in the ancestry assignment. High probability = confident, probability near 0.5 = uncertain.

### Visualization

In [ ]:
par(mfrow = c(4, 1), mar = c(4, 4, 2, 1))

# Panel 1: Observed haplotype
plot(1:n_markers, observed_haplotype, type = "h", lwd = 3, col = "darkgray",
     xlab = "Marker position", ylab = "Allele", 
     main = "Panel A: Observed Admixed Haplotype",
     ylim = c(-0.1, 1.1))
text(5, 0.9, "Allele values\n(0 or 1)", adj = 0, cex = 0.8)

# Panel 2: Allele frequency comparison
plot(1:n_markers, pop1_freq, type = "l", lwd = 2, col = "blue", 
     ylim = c(0, 1), xlab = "Marker position", 
     ylab = "Allele frequency",
     main = "Panel B: Population Allele Frequencies")
lines(1:n_markers, pop2_freq, lwd = 2, col = "red")
legend("topright", legend = c("Population 1", "Population 2"), 
       col = c("blue", "red"), lwd = 2)
text(5, 0.5, sprintf("Mean difference: %.2f\n(Larger = easier inference)", 
                     mean(pop2_freq - pop1_freq)), adj = 0, cex = 0.8)

# Panel 3: True ancestry
plot(1:n_markers, true_ancestry, type = "s", lwd = 4, col = "darkgreen",
     xlab = "Marker position", ylab = "Ancestry", 
     main = "Panel C: True Local Ancestry (What We Want to Infer)",
     ylim = c(0.5, 2.5), yaxt = "n")
axis(2, at = 1:2, labels = c("Pop 1", "Pop 2"))
abline(h = 1.5, lty = 2, col = "gray")
grid()
# Add ancestry block labels
text(5, 1.2, "Pop 1", cex = 0.9)
text(20, 2.2, "Pop 2", cex = 0.9)
text(35, 1.2, "Pop 1", cex = 0.9)
text(44, 2.2, "Pop 2", cex = 0.9)
text(49, 1.2, "Pop 1", cex = 0.9)

# Panel 4: Inferred ancestry with uncertainty
plot(1:n_markers, posterior_ancestry[, 1], type = "l", lwd = 3, col = "blue",
     xlab = "Marker position", ylab = "P(Population 1)", 
     main = "Panel D: Inferred Local Ancestry (HMM Posterior Probability)",
     ylim = c(0, 1))
lines(1:n_markers, posterior_ancestry[, 2], lwd = 3, col = "red")
abline(h = 0.5, lty = 2, col = "black", lwd = 2)
legend("topright", 
       legend = c("P(Pop 1)", "P(Pop 2)", "Decision boundary"), 
       col = c("blue", "red", "black"), lwd = c(3, 3, 2), lty = c(1, 1, 2))
# Shade regions by most likely ancestry
polygon(c(1:n_markers, n_markers:1), 
        c(posterior_ancestry[, 1], rep(0, n_markers)),
        col = rgb(0, 0, 1, 0.2), border = NA)
polygon(c(1:n_markers, n_markers:1), 
        c(posterior_ancestry[, 2], rep(1, n_markers)),
        col = rgb(1, 0, 0, 0.2), border = NA)
text(5, 0.9, "High confidence\nin Pop 1", adj = 0, cex = 0.8)
text(20, 0.1, "High confidence\nin Pop 2", adj = 0, cex = 0.8)